In [ ]:
pip install streamlit

In [ ]:
import os   #let's us to interact with computer OS. eg, create/delete folders,read files,check file paths etc...
import io   #stands for i/p o/p .Let's us work with data as if it's a file(even if it's in a memory)
import re   #Regular expression. helps to find or replace pattern in text
import json  #used to work with JSON data(common format to store data like a dict)
import pandas as pd  #pandas-->data handling module, popular library for data analysis,works with table(rows and column) like excel
import numpy as np  #numpy-->data handling module,library for numerical calculation,great for working with arrays,matrics and math operation
import time  #let's us work with time(pause pgm,measure how long something takes)
import streamlit as st  #stramlit-->app & dashboard,used to make web apps for DS & ML
from typing import List,Dict,Tuple,Any  #type hinting(not code execution,just for clarity),helps to explain datatypes in function

e.g for type hinting

def add_numbers(numbers: List[int]) -> int:
   return sum(numbers)

list[int] means the input is a list of integer
-> int means the function returns as integer

in cursor sentence-transformers are not defaulty installed,so use !pip install sentence-transformers

In [ ]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import faiss
from pypdf import PdfReader
from docx import Document as DocxDocument

NLP & AI models
1.from sentence_transformers import SentenceTransformer
   library to create sentence embbedding(turns sentence into num vector);useful for sementaic search,recommandation,clustering
   Example:
   "I love cats" → [0.12, -0.55, 0.88, ...] (vector).

2.from transformers import pipeline
   Hugging Face Transformers library.
   pipeline is a shortcut to use pre-trained AI models easily.
   Example tasks: text summarization, translation, sentiment analysis, question answering.
   Example:
   from transformers import pipeline
   summarizer = pipeline("summarization")
   print(summarizer("I love tom"))

Searching Similar Data
1.import faiss
  FAISS (by Facebook/Meta) is for fast similarity search.
  Helps you search quickly through millions of vectors (like sentence embeddings).
  Example: find the most similar sentence/document to a query.

Reading Files
1.from pypdf import PdfReader
  Library to read text from PDF files.
  Example: load a PDF, get number of pages, extract text.

from docx import Document as DocxDocument
Library to read and write Word documents (.docx).
Example: open a .docx file and read paragraphs, or create a new Word file.


In short:

SentenceTransformer → turns sentences into numbers (vectors).
pipeline → quick way to use AI models (summarize, translate, classify).
faiss → finds similar vectors really fast (used for search).
PdfReader → extracts text from PDFs.
DocxDocument → reads/writes Word docs.

In [ ]:
pip install faiss-cpu

In [ ]:
pip install pypdf

In [ ]:
pip install python-docx

In [ ]:
!pip install sentence-transformers

In [ ]:
#clean & chunk text
def clean_text(text:str)->str:
  text=re.sub(r"\s+"," ",text)
  return text.strip()

What it does step by step:

def clean_text(text: str) -> str:
Defines a function called clean_text.
It takes some text as input (text).
-> str means it will return a string.

re.sub(r"\s+", " ", text)
Uses regular expressions (re).
\s+ means "one or more spaces, tabs, or newlines".
Replaces them with a single space " ".
Example: "Hello World\n\nHow are you?" → "Hello World How are you?".

text.strip()
Removes extra spaces at the start and end of the text.
Example: " Hello World " → "Hello World".

In [ ]:
def chunk_text(text:str, chunk_size:int=800, overlap:int=120) -> List[str]:
  """
  Character based chunking(simple & Robust)
  Chunk size ~800 chars works well for small model like FLAN-T5
  """
  text = clean_text(text)
  chunks = []
  start = 0
  n = len(text)
  while start < n:
    end = min(start + chunk_size, n)
    chunk = text[start:end]
    chunks.append(chunk)
    start=end-overlap
    if start < 0:
      start = 0
  return chunks

What it does

This function splits long text into smaller pieces (chunks).
Inputs
text: str → the text you want to split.
chunk_size: int = 800 → how many characters per chunk (default 800).
overlap: int = 120 → how many characters overlap between chunks (default 120).
Returns: a list of text chunks (List[str]).

text = clean_text(text)
Cleans the text (removes extra spaces/newlines).

Initialize variables
chunks = [] → empty list to store the text pieces.
start = 0 → where we start cutting.
n = len(text) → total number of characters in text.

Loop until we reach the end of the text
end = min(start + chunk_size, n)
→ defines where the chunk ends.
chunk = text[start:end]
→ takes a slice of the text.
chunks.append(chunk)
→ saves the chunk in the list.

Overlap handling
start = end - overlap
→ moves the start backward by 120 chars so the next chunk overlaps with the previous one.
This is important because AI models may lose context if chunks don’t overlap.

Return chunks
After looping, we return the list of all chunks.

In [ ]:
#file loader(PDF,DOCX,CSV,TXT)

In [ ]:
def load_txt(file_bytes: bytes)->str:
  return file_bytes.decode("utf-8",errors="ignore")

What it does

file_bytes: bytes
The function takes a file that’s been read in bytes form (raw computer data).
Example: when you upload a .txt file, it is often read as bytes first.

.decode("utf-8", errors="ignore")
Converts those bytes into a human-readable string using the utf-8 text format (the most common text encoding).
errors="ignore" means: if there are weird symbols it can’t decode, just skip them instead of crashing.

Returns
A string version of the file’s contents.

In [ ]:
def load_pdf(file_bytes: bytes)->str:
  with io.BytesIO(file_bytes) as fb:
    reader = PdfReader(fb)
    texts=[]
    for page in reader.pages:
      try:
        t=page.extract_text() or ""
      except Exception :
        t=""
      if t:
        texts.append(t)
  return "\n".join(texts)

What it does

This function takes a PDF file in bytes and extracts all the text inside it.
Step by step:
with io.BytesIO(file_bytes) as fb:
Turns the raw file bytes into a file-like object (fb) so that PdfReader can read it (like opening a real file).

reader = PdfReader(fb)
Creates a PdfReader object to work with the PDF.

texts = []
An empty list to collect text from each page.

Loop through each page
for page in reader.pages:
Goes page by page inside the PDF.

Extract text safely
try:
    t = page.extract_text() or ""
except Exception:
    t = ""
page.extract_text() → tries to pull text from the page.
or "" → if it returns None, replace with an empty string.
except → if extraction fails (like scanned PDFs with images), it just skips.

Add text if available
if t:
    texts.append(t)
Saves the text from that page into the list.

Return joined text
return "\n".join(texts)
Combines all page texts into one big string, separated by newlines.

In [ ]:
def load_docx(file_bytes: bytes)->str:
  with io.BytesIO(file_bytes) as fb:
    doc=DocxDocument(fb)
  return "\n".join(p.text for p in doc.paragraphs)

This function takes a Word file (.docx) in bytes and extracts all the text inside it.

Step by step:
with io.BytesIO(file_bytes) as fb:
Turns the raw file bytes into a file-like object (fb), so Python can treat it like an actual .docx file.

doc = DocxDocument(fb)
Opens the Word document using the python-docx library.

doc.paragraphs
Gives you a list of all the paragraphs in the Word document.
Each p is a paragraph object, and p.text is its text content.

"/n".join(p.text for p in doc.paragraphs)
Goes through each paragraph (p.text) and joins them together into one big string.

In [ ]:
def load_csv(file_bytes: bytes)->str:
  with io.BytesIO(file_bytes) as fb:
    df=pd.read_csv(fb)
    #converting to readable FAQ-like table text
  return df.to_csv(index=False)

This function takes a CSV file in bytes and converts it into a text format (CSV string).

Step by step:
with io.BytesIO(file_bytes) as fb:
Converts the raw bytes into a file-like object (fb) so pandas can read it.

df = pd.read_csv(fb)
Reads the CSV file into a pandas DataFrame (like an Excel table in Python).

Comment: # converting to readable FAQ-like table text
This means the idea is to convert the data into a nice text format (like Q&A style), but in this code they just output it back as CSV text.

return df.to_csv(index=False)
Converts the DataFrame back into a CSV string, without the row index.
Basically, it gives you the CSV contents as plain text.

In [ ]:
def read_any(file)->Tuple[str,str]:
  name=file.name.lower()
  content=file.read()
  if name.endswith(".pdf"):
    return "pdf",load_pdf(content)
  elif name.endswith(".docx"):
    return "docx",load_docx(content)
  elif name.endswith(".csv"):
    return "csv",load_csv(content)
  elif name.endswith(".txt"):
    return "txt",load_txt(content)
  else:
    raise ValueError("Unsupported file type.Please upload PDF,DOCX,TXT, OR CSV.")

This function can read different types of files (PDF, DOCX, CSV, TXT) using the right loader function.

Step by step:
name = file.name.lower()
Gets the file name (like "example.PDF").
Converts it to lowercase so it’s easier to check extensions (e.g., "pdf" vs "PDF").

content = file.read()
Reads the file content into memory (as bytes).

Check the file type using extension
If the file ends with .pdf → call load_pdf(content)
If .docx → call load_docx(content)
If .csv → call load_csv(content)
If .txt → call load_txt(content)

Return a tuple
First item = file type (like "pdf", "docx", etc.).
Second item = the extracted text from the file.

Unsupported file
If the file type isn’t recognized, it raises an error telling the user only PDF, DOCX, TXT, or CSV are allowed.

In [ ]:
#Embedding + FAISS handling
@st.cache_resource
def get_embedder():
  return SentenceTransformer("all-MiniLM-L6-v2")

What it does

@st.cache_resource
This is a Streamlit decorator.
It tells Streamlit:
“Run this function only once and cache (remember) the result.”
So if you call get_embedder() many times in the app, it won’t reload the model every time (saves time).

def get_embedder():
Defines a function named get_embedder.

SentenceTransformer("all-MiniLM-L6-v2")
Loads a pretrained embedding model from sentence-transformers.
Model name: all-MiniLM-L6-v2 (a small, fast, but good model for embeddings).
This model converts sentences into embeddings (numeric vectors).
Example: "Hello world" → [0.23, -0.51, 0.88, ...]

return SentenceTransformer(...)
The function returns the model object.
So now, whenever you need embeddings, you just call:
embedder = get_embeder()
vectors=embedder.encode(["hi sam","how u doing?"])

summary:
This function loads a text embedding model once, caches it, and reuses it.
The model converts sentences → vectors (embeddings).
These embeddings are later used with FAISS for similarity search.

In [ ]:
def build_or_load_index(
    embedder:SentenceTransformer,
    storage_dir:str="storage"
) ->Tuple[faiss.IndexFlatL2,List[dict[str,Any]]]:
    os.makedirs(storage_dir,exist_ok=True)
    index_path=os.path.join(storage_dir,"faiss.index")
    meta_path=os.path.join(storage_dir,"meta.npy")

    if os.path.exists(index_path) and os.path.exists(meta_path):
        index=faiss.read_index(index_path)
        metadata=np.load(meta_path,allow_pickle=True).tolist()
    return index,metadata

    #Empty new index
    index=faiss.IndexFlatL2(384)
    metadata: List[Dict[str,Any]]=[]
    return index,metadata


This function either loads an existing FAISS index (and metadata) from disk or, if none exists, creates a new empty index.

Step by step:
Function definition
Takes in an embedder (the SentenceTransformer model).
storage_dir="storage" → default folder where index files are saved.
Returns:
a FAISS index (faiss.IndexFlatL2)
metadata (list of dictionaries with extra info about each chunk).

Paths for storage
index_path = os.path.join(storage_dir, "faiss.index")
meta_path  = os.path.join(storage_dir, "meta.npy")

One file for the FAISS index.
One file for the metadata.

Check if files exist
if os.path.exists(index_path) and os.path.exists(meta_path):
    index = faiss.read_index(index_path)        # load FAISS index
    metadata = np.load(meta_path, allow_pickle=True).tolist()  # load metadata
    return index, metadata
If both files are present → load them from disk and return.

If no files found → create a new index
index = faiss.IndexFlatL2(384)
metadata: List[Dict[str, Any]] = []
return index, metadata
Creates a new FAISS index with vectors of size 384 (because "all-MiniLM-L6-v2" embeddings are 384-dimensional).
Starts with an empty metadata list.

Example usage
embedder = get_embedder()
index, metadata = build_or_load_index(embedder)

print(index.ntotal)   # number of vectors stored
print(len(metadata))  # number of metadata entries


If nothing exists yet → you get an empty FAISS index.
If files exist → it resumes from saved data.

In simple terms:
This function makes sure you always have a FAISS index ready:
If you’ve already built and saved one → it loads it.
If not → it creates a new, empty one.

What is a FAISS index?

FAISS = Facebook AI Similarity Search.
A FAISS index is like a special database designed to store vectors (embeddings) and let you quickly find the ones that are most similar to a query.

Think of it as:
You turn sentences into number vectors using SentenceTransformer.
You put those vectors inside a FAISS index.
Later, when you give a new query (also turned into a vector), FAISS searches the index and finds the closest vectors (i.e., most similar sentences).
So instead of searching text directly, you’re searching based on semantic meaning.

What is faiss.IndexFlatL2?

FAISS provides different types of indexes.
IndexFlatL2 means:
Flat → It stores all vectors directly (no fancy compression, no approximation).
L2 → It measures similarity using L2 distance (Euclidean distance).
That’s just the usual "straight-line distance" in multi-dimensional space.

sumaary:
FAISS index = a smart database for embeddings.
IndexFlatL2 = the simplest type of FAISS index, which stores vectors and compares them using Euclidean distance to find similar ones.

Later, you can use more advanced FAISS indexes (IndexIVF, HNSW, etc.) for faster searches when you have millions of vectors.

In [ ]:
def persist_index(index: faiss.IndexFlatL2, metadata: List[Dict[str, Any]], storage_dir: str = "storage"):
    os.makedirs(storage_dir, exist_ok=True)
    faiss.write_index(index, os.path.join(storage_dir, "faiss.index"))
    np.save(os.path.join(storage_dir, "meta.npy"), np.array(metadata, dtype=object), allow_pickle=True)

This function saves (persists) the FAISS index and metadata to disk so that you can use them later without rebuilding from scratch.

Step by step

Create storage folder if missing
os.makedirs(storage_dir, exist_ok=True)
Makes sure the folder exists (e.g., "storage").
If it already exists → no error (because of exist_ok=True).

Save the FAISS index
faiss.write_index(index, os.path.join(storage_dir, "faiss.index"))
Writes the FAISS index (all the vectors) to a file called faiss.index.

Save the metadata
np.save(os.path.join(storage_dir, "meta.npy"), np.array(metadata, dtype=object), allow_pickle=True)
Converts metadata (Python list of dictionaries) into a NumPy array.
Saves it into meta.npy.
allow_pickle=True allows saving Python objects (like dictionaries).

In simple terms
Think of it like saving your progress in a game:
The FAISS index is like the game’s world data (your embeddings).
The metadata is like the notes telling which vector belongs to which text.
This function writes both into files so next time you run the program, you can just load them back instead of starting over.

In [ ]:
def add_texts_to_index(
    texts: List[str],
    source_name: str,
    embedder: SentenceTransformer,
    index: faiss.IndexFlatL2,
    metadata: List[Dict[str,Any]]
):
    if not texts:
      return
    vectors = embedder.encode(texts,convert_to_numpy=True,normalize_embeddings=False)
    index.add(vectors)
    for i,t in enumerate(texts):
      metadata.append({
          "source": source_name,
          "chunk_id": i,
          "text": t
      })

This function takes text chunks, turns them into embeddings, stores them in FAISS, and records metadata.

Step by step

Check if texts exist
if not texts:
    return
If there are no texts, just stop.

Convert texts → vectors
vectors = embedder.encode(texts, convert_to_numpy=True, normalize_embeddings=False)
Uses the SentenceTransformer model to convert each text chunk into a numeric vector (embedding).
Example: "Hello world" → [0.12, -0.34, 0.56, ...]

Add vectors to FAISS index
index.add(vectors)
Stores those embeddings into the FAISS index for similarity search.

Save metadata for each chunk
for i, t in enumerate(texts):
    metadata.append({
        "source": source_name,
        "chunk_id": i,
        "text": t
    })
Keeps track of:
source_name → which file the text came from
chunk_id → position number of the chunk
text → the actual chunk of text

summary.
This function is like filing documents in a library:
First, it turns the text into a special code (vector) the computer can understand.
Then it stores the code in FAISS (like putting it in a drawer).
At the same time, it writes a note card (metadata) saying “this code belongs to file X, chunk Y, with this text.”
So later, when you search FAISS for similar text, you can also look up the original chunk and where it came from.

In [ ]:
#Retriever
def retriever(query: str, embedder,index,metadata,k: int=3):
  if index.ntotal == 0:
    return []
  qvec = embedder.encode([query],convert_to_numpy=True)
  D, I = index.search(qvec, k=min(k, index.ntotal))
  results=[]
  for idx in I[0]:
    md = metadata(idx)
    results.append(md)
  return results

This function takes a user’s query, finds the most similar text chunks in the FAISS index, and returns their metadata.

Step by step

Check if index is empty
if index.ntotal == 0:
    return []
If no data has been added yet, just return an empty list.

Convert query → embedding
qvec = embedder.encode([query], convert_to_numpy=True)
Turns your search question into a vector (same way we did with the text chunks).
Example: "What is AI?" → [0.22, -0.15, 0.87, ...]

Search FAISS for nearest neighbors
D, I = index.search(qvec, k=min(k, index.ntotal))
index.search finds the k most similar vectors in FAISS.
D = distances (how close each result is)
I = indices (positions of matching chunks in the metadata list)

Collect metadata for results

for idx in I[0]:
    md = metadata[idx]
    results.append(md)


Uses the indices I to fetch the corresponding metadata (original text, source file, chunk ID).
Builds a list of result dictionaries.
Return the matches
return results
Final output is a list of the most relevant text chunks.

summary
This function is like asking the library a question:
You phrase your question (query).
The system translates it into the same “special code” as the stored chunks.
FAISS finds the chunks that look closest to your query in vector space.
It then gives you the original text + source info (from metadata).

 Example:
results = retrieve("What is AI?", embedder, index, metadata, k=2)

Might return:

[
  {"source": "notes.pdf", "chunk_id": 0, "text": "Artificial Intelligence is..."},
  {"source": "ai_book.docx", "chunk_id": 3, "text": "AI means creating machines..."}
]


In [ ]:
#Generator

In [ ]:
@st.cache_resource
def get_generator():
  return pipeline("text2text-generation",model="google/flan-t5-base") #flan-t5 is light and fine for grounded ans.

In [ ]:
def synthesize_answer(user_query: str,contexts: List[Dict[str,Any]],history: List[Dict[str,str]],generator) -> str:
  conv = "\n".join([f"{m['role'].capitalize()}:{m['content']}" for m in history[-6:]])
  context_text = "\n\n".join([f"[{i+1}] {c['text']}" for i, c in enumerate(contexts)]) if contexts else "No context"
  prompt = f"""You are a helpful, concise support assistant. Answer the final user question using ONLY the provided context. If missing, say you don't know.
Coversation so far:
{conv}

context:
{context_text}

Final user question: {user_query}
Answer:"""
  out = generator(prompt,max_length=220,num_return_sequences=1)[0]["generated_text"]
  return out.strip()

This function takes the user’s question, adds context from documents, adds chat history, and asks the AI model (FLAN-T5) to generate an answer.

Step by step
1. Get the AI model
return pipeline("text2text-generation", model="google/flan-t5-base")
Uses FLAN-T5, a lightweight model good at answering questions with context.
pipeline("text2text-generation") means: given a text prompt, generate an output text.

2. Collect conversation history
convo = "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in history[-6:]])
Formats the last 6 messages from the conversation.
Example:
User: What is AI?
Assistant: AI means...

3. Collect relevant context
context_text = "\n\n".join([f"[{i+1}] {c['text']}" for i, c in enumerate(contexts)]) if contexts else "No context."
Joins together the retrieved chunks (from FAISS).
Example:
[1] Artificial Intelligence is the field of...
[2] AI can be used in...

4. Build the prompt
prompt = f"""You are a helpful, concise support assistant...
Combines:
The conversation history (convo)
The context from documents (context_text)
The user’s latest question (user_query)
It also instructs the model:
“Use ONLY the context. If not found, say you don’t know.”

5. Generate the answer
out = generator(prompt, max_length=220, num_return_sequences=1)[0]["generated_text"]
Sends the prompt to the model (generator).
Limits answer length to 220 tokens.
Returns one best answer.

6. Return clean output
return out.strip()
Strips extra spaces/newlines.

summary
This function is like the final stage of a Q&A assistant:
Gather what the user asked before (chat history).
Gather useful text from the documents (context).
Create a clear instruction prompt.
Ask the AI model (FLAN-T5) to generate a short, helpful answer.
Return that as the assistant’s reply.

Example:
query = "What is AI?"
contexts = [{"text": "Artificial Intelligence is the science of making machines smart."}]
history = [{"role": "user", "content": "Hello"}, {"role": "assistant", "content": "Hi, how can I help?"}]
generator = get_generator()

print(synthesize_answer(query, contexts, history, generator))

Might output:
"AI is the science of making machines smart, as described in the context."

In [ ]:
#streamlit app
st.set_page_config(page_title="AI Support Assistant (Multi-doc, Local)",page_icon="🤖",layout="wide")
st.title("🤖 AI-Powered Customer Support Assistant")
st.caption("Runs locally • No API keys • Multi-document RAG • Citations • Session memory")

missing ScriptRunContext! This warning can be ignored when running in bare mode.
DeltaGenerator()
is from Streamlit.

Let me explain in simple terms:
Why it happens
Streamlit code (like st.title(), st.caption(), st.write(), etc.) is meant to run inside a Streamlit app (when you launch it using streamlit run your_app.py).

If you just run the script in a normal Python environment (like Jupyter Notebook, Colab, or plain python script.py), Streamlit doesn’t have its "context" (the special environment it needs to show things in the web app).

That’s why it prints this warning: “missing ScriptRunContext! This warning can be ignored when running in bare mode.”

In plain English → Streamlit is saying: “Hey, you’re not running me in app mode, so I can’t actually show UI. I’ll just return an empty object instead.”

How to fix / use properly
If you actually want to see the app UI, you need to run:
streamlit run your_app.py
This will open a local web page (usually at http://localhost:8501) with your assistant interface.

Inside Jupyter/Colab, Streamlit calls will not render the UI — they just return placeholders (DeltaGenerator() objects).

In simple terms
The warning is not an error.
It just means: “I’m in notebook/python mode, not in Streamlit app mode.”
To really use the chatbot app, you must run it with Streamlit.

In cursor ai app, first i given pip install streamlit in this cursor ai terminal, it succefully executed but when i give import streamlit as st, it's showing error

I can see what went wrong.
You tried this in the terminal:
import streamlit as st
But that is Python code, not a terminal command. That’s why PowerShell is saying:
import : The term 'import' is not recognized...

Correct way:

First, make sure Streamlit is installed (you already did this):
pip install streamlit
Then, instead of running import streamlit as st in the terminal,
you need to run your app with Streamlit like this:
streamlit run chatbot_st.py
(replace chatbot_st.py with your filename)

That will open a local webpage in your browser (usually http://localhost:8501) with your app UI.

What not to do
Don’t type import streamlit as st in the terminal — it only works inside your Python script.

Example inside chatbot_st.py:
import streamlit as st
st.title("🤖 AI-Powered Customer Support Assistant")
Try running:
streamlit run chatbot_st.py

PS C:\Users\Admin\Documents\CusorAIDoc> streamlit run .\chatbot_st.py streamlit : The term 'streamlit' is not recognized as the name of a cmdlet, function, script file, or operable program. Check the spelling of the name, or if a path was included, verify that the path is correct and try again. At line:1 char:1 + streamlit run .\chatbot_st.py + ~~~~~~~~~ + CategoryInfo : ObjectNotFound: (streamlit:String) [], CommandNotFoundException + FullyQualifiedErrorId : CommandNotFoundException


PowerShell can’t find the streamlit command, which means your PATH/environment is not set up correctly.
Even though pip install streamlit worked, the streamlit.exe script is not in your PATH.

Step 1: Verify installation
Run this in the same terminal:
python -m pip show streamlit
If it shows details (version, location, etc.), then Streamlit is installed, but the command shortcut (streamlit) is not linked to PATH.

Step 2: Run Streamlit without PATH issues
Instead of streamlit run ..., run it through Python directly:
python -m streamlit run chatbot_st.py
That forces Python to use the installed Streamlit module.

Step 3 (Optional Fix for PATH)
If you want the streamlit command to work globally:
Find where scripts are installed:
python -m site --user-base
It will return a path like:
C:\Users\Admin\AppData\Roaming\Python\Python313
Inside it, there’s a Scripts\ folder containing streamlit.exe.
Add that Scripts\ folder to your PATH in Windows environment variables.

For now, the quickest way is:
python -m streamlit run chatbot_st.py


Do you want me to walk you through how to permanently fix the PATH so you can just type streamlit run ... next time?

In [ ]:
#don't execute it
st.set_page_config(
    page_title="AI Support Assistant (Multi-doc, Local)",
    page_icon="🤖",
    layout="wide"
)
st.title("🤖 AI-Powered Customer Support Assistant")
st.caption("Runs locally • No API keys • Multi-document RAG • Citations • Session memory")


What each line does

st.set_page_config(...)
Configures the overall page in Streamlit.
page_title="AI Support Assistant (Multi-doc, Local)" → The title shown in the browser tab.
page_icon="🤖" → The favicon (little emoji in tab bar).
layout="wide" → Makes the app stretch to the full width of the screen (instead of narrow center).

st.title("🤖 AI-Powered Customer Support Assistant")
Displays a big headline at the top of the app.
In this case:
🤖 AI-Powered Customer Support Assistant

st.caption("Runs locally • No API keys • Multi-document RAG • Citations • Session memory")
Adds a smaller subtitle / description under the title.
Tells the user key features of the app:
Runs locally (no internet needed for model calls)
No API keys required
Can handle multiple documents (RAG = Retrieval Augmented Generation)
Gives citations (where the answer came from)
Remembers session chat history

summary:
This block is just about making your app look professional when you open it:
Sets tab title + icon
Shows a big app title
Shows a short description of what it can do

In [1]:
#don't execute
# Side: model & index controls
with st.sidebar:
    st.subheader("📁 Knowledge Base")
    st.write("Upload PDFs, DOCX, TXT, CSV. They’ll be embedded locally and indexed.")

    uploaded_files = st.file_uploader(
        "Add documents",
        type=["pdf", "docx", "txt", "csv"],
        accept_multiple_files=True
    )
    paste_text = st.text_area("Or paste text (optional):", height=120, placeholder="Paste product manuals, FAQs, SOPs...")

    chunk_size = st.slider("Chunk size (chars)", 400, 1500, 800, 50)
    overlap = st.slider("Overlap (chars)", 50, 400, 120, 10)
    top_k = st.slider("Citations (top-K)", 1, 5, 3, 1)


    # Load components
    embedder = get_embedder()
    generator = get_generator()
    index, metadata = build_or_load_index(embedder)

    if st.button("➕ Ingest now", use_container_width=True):
        new_texts = []

        # From files
        for f in uploaded_files or []:
            try:
                ftype, raw = read_any(f)
                chunks = chunk_text(raw, chunk_size=chunk_size, overlap=overlap)
                new_texts.extend([(f.name, c) for c in chunks])
            except Exception as e:
                st.error(f"Failed to read {f.name}: {e}")

        # From pasted text
        if paste_text.strip():
            chunks = chunk_text(paste_text, chunk_size=chunk_size, overlap=overlap)
            new_texts.extend([("pasted_text", c) for c in chunks])

        if new_texts:
            # Batch by source for cleaner metadata
            by_source: Dict[str, List[str]] = {}
            for src, c in new_texts:
                by_source.setdefault(src, []).append(c)

            total_added = 0
            for src, chunks in by_source.items():
                add_texts_to_index(chunks, src, embedder, index, metadata)
                total_added += len(chunks)

            persist_index(index, metadata)
            st.success(f"Ingested {total_added} chunks from {len(by_source)} sources 👍")
        else:
            st.info("No new content to ingest.")

    if st.button("🧹 Reset index", type="secondary", use_container_width=True):
        # wipe storage
        try:
            os.remove(os.path.join("storage", "faiss.index"))
        except FileNotFoundError:
            pass
        try:
            os.remove(os.path.join("storage", "meta.npy"))
        except FileNotFoundError:
            pass
        st.experimental_rerun()

    st.markdown("---")
    st.write("**Index size:**", getattr(index, "ntotal", 0))
    if os.path.exists("data/faqs.txt"):
        st.write("Tip: add your starter FAQs in `data/faqs.txt` and ingest them too.")

# ---- Conversation state ----
if "history" not in st.session_state:
    st.session_state.history = []

# Show chat history
for msg in st.session_state.history:
    st.chat_message(msg["role"]).markdown(msg["content"])

# Chat input
user_query = st.chat_input("Ask about your product/service, refunds, onboarding, SLAs, etc.")
if user_query:
    st.session_state.history.append({"role": "user", "content": user_query})
    st.chat_message("user").markdown(user_query)

    # Retrieve
    contexts = retriever(user_query, embedder, index, metadata, k=top_k)

    # Generate grounded answer
    answer = synthesize_answer(user_query, contexts, st.session_state.history, generator)
    st.session_state.history.append({"role": "assistant", "content": answer})

    # Display answer
    st.chat_message("assistant").markdown(answer)

    # Show citations panel
    if contexts:
        with st.expander("🔎 Sources"):
            for i, c in enumerate(contexts, 1):
                st.markdown(f"**[{i}] {c['source']} — chunk {c['chunk_id']}**")
                st.markdown(f"> {c['text'][:400]}{'…' if len(c['text'])>400 else ''}")
    else:
        st.info("No sources found yet. Ingest documents in the left panel.")


NameError: name 'st' is not defined

This code is creating a sidebar in your Streamlit app where the user can control the knowledge base and how documents are processed.

Line by line:
with st.sidebar:
Everything inside this block will appear on the sidebar of the app (left side).

st.subheader("📁 Knowledge Base")
st.write("Upload PDFs, DOCX, TXT, CSV. They’ll be embedded locally and indexed.")
Adds a title and a description to explain what the sidebar is for.
It tells the user they can upload documents that will be turned into embeddings and indexed (so the AI can search them).

uploaded_files = st.file_uploader(
    "Add documents",
    type=["pdf", "docx", "txt", "csv"],
    accept_multiple_files=True
)

This creates a file uploader button.
Users can upload multiple files (PDF, Word, TXT, CSV).
Those files will be stored in uploaded_files.

paste_text = st.text_area("Or paste text (optional):", height=120, placeholder="Paste product manuals, FAQs, SOPs...")
This gives another option: instead of uploading files, users can paste raw text (like FAQs or manuals).
That text will go into paste_text.

chunk_size = st.slider("Chunk size (chars)", 400, 1500, 800, 50)
Chunk size slider:
When we embed documents, they’re too long for the model to handle at once.
So we break them into smaller pieces ("chunks").
This slider lets the user pick the chunk size (default = 800 characters, min = 400, max = 1500).

overlap = st.slider("Overlap (chars)", 50, 400, 120, 10)
Overlap slider:
When splitting text into chunks, some text is repeated between chunks (overlap).
This helps the AI maintain context between chunks.
Here, default is 120 characters overlap.

top_k = st.slider("Citations (top-K)", 1, 5, 3, 1)
Top-K slider:
This controls how many top relevant chunks (citations) are retrieved when answering a query.
For example, if top_k=3, it will pull the 3 most relevant text chunks to answer.

In summary:

This sidebar is like a control panel where the user can:
Upload files 📂
Paste text ✍️
Adjust how documents are split (chunk size + overlap)
Control how many citations to show in answers

From load components to wipe storage -> This is the part of your app that actually loads AI models, ingests documents, and manages the index.

Step by Step:
embedder = get_embedder()
generator = get_generator()
index, metadata = build_or_load_index(embedder)

Loads:
embedder = model that converts text into embeddings (vectors).
generator = model that answers questions.
index, metadata = either builds a new FAISS index (vector database) or loads an existing one.

if st.button("➕ Ingest now", use_container_width=True):
When user clicks “Ingest now”, we start processing new documents or pasted text

📂 From uploaded files:
for f in uploaded_files or []:
    try:
        ftype, raw = read_any(f)
        chunks = chunk_text(raw, chunk_size=chunk_size, overlap=overlap)
        new_texts.extend([(f.name, c) for c in chunks])
    except Exception as e:
        st.error(f"Failed to read {f.name}: {e}")

For each uploaded file:
Read its content (read_any).
Break into chunks (chunk_text).
Add chunks to new_texts.
  If something goes wrong, show an error.

  ✍️ From pasted text:
if paste_text.strip():
    chunks = chunk_text(paste_text, chunk_size=chunk_size, overlap=overlap)
    new_texts.extend([("pasted_text", c) for c in chunks])

If user typed/pasted something in the text area:
Break into chunks.
Add them as "pasted_text" source.

🗂 Organize by source:
by_source: Dict[str, List[str]] = {}
for src, c in new_texts:
    by_source.setdefault(src, []).append(c)
Groups chunks by file name (or "pasted_text").
Example:
manual.pdf → [chunk1, chunk2, ...]
faq.docx → [chunk1, chunk2, ...]

Add to index:
for src, chunks in by_source.items():
    add_texts_to_index(chunks, src, embedder, index, metadata)
    total_added += len(chunks)
For each source:
Convert chunks into embeddings using embedder.
Store them in index along with metadata.

Save index:
persist_index(index, metadata)
st.success(f"Ingested {total_added} chunks from {len(by_source)} sources 👍")
Saves the FAISS index + metadata to disk.
Shows success message.

❌ If no new content:
else:
    st.info("No new content to ingest.")
Tells user nothing was added.

🧹 Reset index:
if st.button("🧹 Reset index", type="secondary", use_container_width=True):
    # wipe storage
    try:
        os.remove(os.path.join("storage", "faiss.index"))
    except FileNotFoundError:
        pass
    try:
        os.remove(os.path.join("storage", "meta.npy"))
    except FileNotFoundError:
        pass
    st.experimental_rerun()

If user clicks “Reset index”:
Deletes stored FAISS index + metadata.
Restarts the app with an empty index.

Show index info:
st.markdown("---")
st.write("**Index size:**", getattr(index, "ntotal", 0))
if os.path.exists("data/faqs.txt"):
    st.write("Tip: add your starter FAQs in `data/faqs.txt` and ingest them too.")
 Shows:
How many embeddings (chunks) are stored (index.ntotal).
A tip: you can preload FAQs in data/faqs.txt.

Conversation state
if "history" not in st.session_state:
    st.session_state.history = []

Show chat history
for msg in st.session_state.history:
    st.chat_message(msg["role"]).markdown(msg["content"])
ChatGPT said:

let’s break this block step by step in simple terms:

if "history" not in st.session_state:
    st.session_state.history = []


This checks if we already have a chat history saved in Streamlit’s session_state (which remembers things while the app is running).

If there’s no "history" yet, it creates one as an empty list [].

This list will store the conversation messages (like user messages and AI replies).


for msg in st.session_state.history:
    st.chat_message(msg["role"]).markdown(msg["content"])


This loop goes through every message saved in the history.

Each msg is a dictionary with two keys:

"role" → who sent it ("user" or "assistant")

"content" → the actual text of the message

Then it uses st.chat_message(...) to display the message in the chat window, and .markdown(...) makes it look nice.

Chat input
user_query = st.chat_input("Ask about your product/service, refunds, onboarding, SLAs, etc.")
if user_query:
    st.session_state.history.append({"role": "user", "content": user_query})
    st.chat_message("user").markdown(user_query)

    Retrieve
    contexts = retrieve(user_query, embedder, index, metadata, k=top_k)

    Generate grounded answer
    answer = synthesize_answer(user_query, contexts, st.session_state.history, generator)
    st.session_state.history.append({"role": "assistant", "content": answer})

    Display answer
    st.chat_message("assistant").markdown(answer)

    Show citations panel
    if contexts:
        with st.expander("🔎 Sources"):
            for i, c in enumerate(contexts, 1):
                st.markdown(f"**[{i}] {c['source']} — chunk {c['chunk_id']}**")
                st.markdown(f"> {c['text'][:400]}{'…' if len(c['text'])>400 else ''}")
    else:
        st.info("No sources found yet. Ingest documents in the left panel.")
ChatGPT said:

Perfect 👍 Let’s break this block into very simple steps.

🔹 What the code does

This is the part of your Streamlit app where the user chats with the AI assistant.

Step by Step:

Take user input

user_query = st.chat_input("Ask about your product/service, refunds, onboarding, SLAs, etc.")


👉 This creates a chat box where the user can type a question.

Save the question in chat history

if user_query:
    st.session_state.history.append({"role": "user", "content": user_query})
    st.chat_message("user").markdown(user_query)


👉 If the user types something:

It gets stored in st.session_state.history (so we can show past messages).

It gets displayed in the chat window.

Find relevant info from documents

contexts = retrieve(user_query, embedder, index, metadata, k=top_k)


👉 This looks inside your uploaded documents to find the most relevant chunks that might answer the question.

Generate an answer using AI

answer = synthesize_answer(user_query, contexts, st.session_state.history, generator)
st.session_state.history.append({"role": "assistant", "content": answer})


👉 It takes:

The user’s question

The retrieved document chunks

The chat history
and creates a grounded answer (AI-generated but based on docs).
Then, the assistant’s reply is also saved into history.

Show the AI’s response

st.chat_message("assistant").markdown(answer)


👉 Displays the assistant’s reply in the chat.

Show sources (citations)

if contexts:
    with st.expander("🔎 Sources"):
        for i, c in enumerate(contexts, 1):
            st.markdown(f"**[{i}] {c['source']} — chunk {c['chunk_id']}**")
            st.markdown(f"> {c['text'][:400]}{'…' if len(c['text'])>400 else ''}")
else:
    st.info("No sources found yet. Ingest documents in the left panel.")


👉 If the assistant used some documents to answer:

It shows them in a collapsible "Sources" panel.

Shows which document (source + chunk ID).

Shows the actual snippet of text it came from.

If no docs were found → it tells the user to upload documents first.

🟢 In super simple words:

User types a question.

App remembers it.

Searches uploaded documents for relevant info.

AI writes an answer based on those docs.

Shows the answer in chat.

Shows where the answer came from (citations).